<a href="https://colab.research.google.com/github/petersawe/PythonQuickStart/blob/main/Headings%2C_Details%2C_Summaries%2C_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
import uuid
#from gemini import Gemini
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.style import WD_STYLE_TYPE
import pathlib
import textwrap
import google.generativeai as genai
from google.colab import userdata
import requests
import io
from PIL import Image
from docx import Document
from docx.shared import Inches
from time import sleep
from google.colab import files

# Configure Google Generative AI
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

# Hugging Face API for generating images
API_URL = "https://api-inference.huggingface.co/models/dataautogpt3/OpenDalleV1.1"
HF_API_KEY = userdata.get('HF_API_KEY')
headers = {"Authorization": HF_API_KEY}

def query_image(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    sleep(1)  # Introducing a 1-second delay
    return response.content

def generate_headings(book_title, book_title_prompt, model):
    response = model.generate_content(book_title_prompt)
    headings = response.text.split(',')
    return headings

def generate_summary(heading, level, book_title, model):
    prompt = f"Summarize the topic of '{heading}' in relation to the '{book_title}' at {level} level"
    summary = model.generate_content(prompt)
    sleep(1)  # Introducing a 1-second delay
    return summary.text

def generate_chapter_details_prompt(heading, summary):
    # Generate chapter details
    details_prompt = f"Generate a detailed explanation of the following chapter, covering key concepts, examples, and practical applications:\nChapter Heading: {heading}\nChapter Summary: {summary}"
    chapter_details = model.generate_content(details_prompt)
    parts = chapter_details.parts
    for part in parts:
        if "text" in part:
            text_content = part.text
            # Process the text content here
    return text_content

def generate_python_code_prompt(heading, summary):
    # Generate Python code example
    code_prompt = f"Provide a Python code example that automates the work or processes described in the following chapter:\nChapter Heading: {heading}\nChapter Summary: {summary}"
    python_code_example = model.generate_content(code_prompt)
    parts = python_code_example.parts
    for part in parts:
        if "text" in part:
            python_code_example = part.text
            # Process the text content here
    return python_code_example

def categorize_headings(headings):
    beginner_headings = []
    intermediate_headings = []
    advanced_headings = []

    for heading in headings:
        if "beginner" in heading.lower():
            beginner_headings.append(heading)
        elif "intermediate" in heading.lower():
            intermediate_headings.append(heading)
        elif "advanced" in heading.lower():
            advanced_headings.append(heading)

    return beginner_headings, intermediate_headings, advanced_headings

import uuid  # Add this import statement

def format_headings(headings):
    formatted_headings = ''
    for heading in headings:
        formatted_headings += f'"{heading.strip()}",\n'
    return formatted_headings



    #style = document.styles['Normal']  # Assuming you have a 'Normal' style
    #style.font = font
    #document.styles.add_style('code', style)



    #if 'code' not in document.styles:
        #style   = document.styles.add_style('code', WD_STYLE_TYPE.PARAGRAPH)



        #document.add_paragraph().style = style

#    code_style = document.styles.get('code', None)

# Define the 'code' style for code blocks
def define_code_style(document):
    # Check if 'code' style already exists
    code_style = None
    for style in document.styles:
        if style.name == 'code':
            code_style = style
            break

    # If 'code' style doesn't exist, add it
    if not code_style:
        code_style = document.styles.add_style('code', WD_STYLE_TYPE.PARAGRAPH)
        font = code_style.font
        font.name = 'Courier New'
        font.size = Pt(10)

    return code_style

#def define_code_style(paragraph):
    # Define the 'code' style for code blocks
#    font = paragraph.runs[0].font
#    font.name = 'Courier New'
#    font.size = Pt(10)
#    paragraph.style.font = font

def add_paragraph_with_style(document, text, style=None):
    # Add a paragraph to the document with the specified style
    paragraph = document.add_paragraph(text)

    if style:
        # Apply the style if specified
        paragraph.stxyle = style

    return paragraph

def create_word_document(headings, level, book_title, model):
    # Generate a unique identifier for the document
    unique_identifier = str(uuid.uuid4())[:8]
    print(f"{unique_identifier} generated")

    # Create a new document for the level
    level_book = Document()
    print(f"New {level_book} created")

    # Apply code style to the chapter document
    define_code_style(level_book)  # Assuming the first paragraph is the code block
    print(f"Code style applied to chapter document ")

    # Add title to the level book
    level_book.add_heading(f'{book_title} - {level} Level', level=0)
    print(f"Title added to the level book")

    # Create a new document for the table of contents
    toc_document = Document()
    print(f"New table of contents created")

    # Apply code style to the chapter document
    define_code_style(toc_document)  # Assuming the first paragraph is the code block
    print(f"Code style applied to table of contents")

    # Add title to the table of contents
    toc_document.add_heading(f'{level} Level Table of Contents', level=1)
    print(f"Title added to table of contents ")

     # Create a new document for the level
    #complete_book = Document()
    #print(f"New document for the level created")

    # Apply code style to the chapter document
    #define_code_style(complete_book)  # Assuming the first paragraph is the code block
    #print(f"Code style applied to  {complete_book} ")

    # Add title to the complete book
    #complete_book.add_heading(book_title, level=0)
    #print(f"Title added to {complete_book} ")

    formatted_headings = format_headings(headings)
    headings_list = formatted_headings.split('\n')
    #print(f"formatted_headings {headings} ")

    # Generate summaries, create chapters, and generate Python code
    for index, heading in enumerate(headings_list, start=1):
        if not heading:
            continue

        heading = heading.strip('",')
        # Generate summary and query image
        summary = generate_summary(heading, level, book_title, model)
        print(f"Summary Generated for {heading} of {book_title} ")
        image_bytes = query_image({"inputs": summary})
        print(f"Image Generated for {heading} of {book_title} ")

        # Generate details prompt
        details_prompt = generate_chapter_details_prompt(heading, summary)
        print(f"Details prompt Generated for {heading} of {book_title} ")

        # Generate Python code prompt
        #python_code_prompt = generate_python_code_prompt(heading, summary)
        #python_code_example = model.generate_content(python_code_prompt).text
        python_code_example = generate_python_code_prompt(heading, summary)
        print(f"Python code prompt Generated for {heading} of {book_title} ")

        # Create a unique identifier for each chapter
        chapter_identifier = f"{index}"
        print(f"Unique identifier {chapter_identifier} generated")

        # Create a new document for each chapter
        chapter_document = Document()
        print(f"New document for the chapter created")

        # Apply code style to the chapter document
        define_code_style(chapter_document)  # Assuming the first paragraph is the code block
        print(f"Code style applied to the chapter document")

        # Add title to the chapter
        chapter_document.add_heading(f'{book_title} - {level} Level', level=1)
        print(f"Title added to {book_title} ")

        # Add heading and summary with proper formatting
        chapter_document.add_heading(heading, level=2)
        print(f"Heading added to {heading}  ")
        chapter_document.add_paragraph(summary)
        print(f"Summary added to {heading}  ")

        # Try to add image to the chapter document
        try:
            image = Image.open(io.BytesIO(image_bytes))
            if image:
                print("Image opened successfully.")
                chapter_document.add_picture(io.BytesIO(image_bytes), width=Inches(2))
                print("Image added to the chapter document.")
            else:
                print("Error: Unable to open the image.")
        except Exception as e:
            print(f"Error opening image: {e}")
            print("Skipping image for this chapter.")

        # Add chapter details
        chapter_document.add_paragraph("Chapter Details:")
        chapter_document.add_paragraph(details_prompt)
        print(f"Chapter details added ")

        # Add Python code example
        #chapter_document.add_paragraph("Python Code Example:")
        add_paragraph_with_style(chapter_document, "Python Code Example:", style='Heading1')

        #chapter_document.add_paragraph(python_code_example, style='code')  # Apply the 'code' style
        add_paragraph_with_style(chapter_document, python_code_example, style='code')
        print(f"Python code example added for {heading} ")

        # Save the chapter document
        try:
            chapter_document.save(f'/content/{level}_{unique_identifier}_{chapter_identifier}.docx')
            print(f"Chapter document saved for: {heading}")
        except FileNotFoundError as e:
            print(f"Error saving chapter document: {e}")
            print("Skipping this chapter.")

        # Add the chapter to the level book
        #level_book.add_heading(f'{level} Level - {heading}', level=1)
        #level_book.add_paragraph(summary)

        # Add heading and summary with proper formatting
        add_paragraph_with_style(level_book, heading, style='Heading1')
        print(f"Heading added to {heading} ")
        add_paragraph_with_style(level_book, summary)
        print(f"Summary added to {heading} of {level_book} ")

        # Try to add image to the level book
        try:
            if image:
                level_book.add_picture(io.BytesIO(image_bytes), width=Inches(2))
        except Exception as e:
            print(f"Error adding image to the level book: {e}")
            print("Skipping image for this chapter in the level book.")

        level_book.add_paragraph()

        # Add chapter details
        level_book.add_paragraph("Chapter Details:")
        level_book.add_paragraph(details_prompt)
        print(f"Chapter details added to {level_book} ")

        # Add Python code example
        #level_book.add_paragraph("Python Code Example:")
        add_paragraph_with_style(level_book, "Python Code Example:", style='Heading1')


        # Split the Python code example into paragraphs
        python_code_paragraphs = python_code_example.split('\n')

        # Apply the 'code' style to each paragraph
        for code_paragraph in python_code_paragraphs:
            level_book.add_paragraph(code_paragraph, style='code')  # Apply the 'code' style

        print(f"Python code examples added for: {level_book} Level ")

        level_book.add_paragraph()

        # Add some space between chapters in the level book
        level_book.add_paragraph()

        # Add the heading to the table of contents
        toc_document.add_heading(f'{index}. {heading}', level=2)
        print(f"Heading added to {heading} ")

        # Save the table of contents document
        try:
            toc_document.save(f'/content/{level}_{unique_identifier}_TOC_{book_title}.docx')
            print(f"{level} Level Table of Contents saved.")
        except FileNotFoundError as e:
            print(f"Error saving table of contents document: {e}")
    # Save the level book
    try:
        level_book.save(f'/content/{level}_{unique_identifier}_{book_title}_level_book.docx')
        print(f"{level} Level Book saved.")
    except FileNotFoundError as e:
        print(f"Error saving level book: {e}")

    # Download the level book
    level_book_path = f'/content/{level}_{unique_identifier}_{book_title}_level_book.docx'
    try:
        files.download(level_book_path)
    except FileNotFoundError as e:
        print(f"Error downloading level book: {e}")

    # Download the table of contents document
    toc_document_path = f'/content/{level}_{unique_identifier}_TOC_{book_title}.docx'
    try:
        files.download(toc_document_path)
    except FileNotFoundError as e:
        print(f"Error downloading table of contents: {e}")

# Generate unique headings for each level
beginner_book_title = 'Beginner\'s Guide to Large Language Models'
beginner_book_title_prompt = f"Create a detailed outline with subtopics, using a comma-separated multilevel list format, for a manual on '{beginner_book_title}'"
beginner_headings = generate_headings(beginner_book_title, beginner_book_title_prompt, model)

intermediate_book_title = 'Intermediate Guide to Large Language Models'
intermediate_book_title_prompt = f"Create a detailed outline with subtopics, using a comma-separated multilevel list format, for a manual on '{intermediate_book_title}'"
intermediate_headings = generate_headings(intermediate_book_title, intermediate_book_title_prompt, model)

advanced_book_title = 'Advanced Guide to Large Language Models'
advanced_book_title_prompt = f"Create a detailed outline with subtopics, using a comma-separated multilevel list format, for a manual on '{advanced_book_title}'"
advanced_headings = generate_headings(advanced_book_title, advanced_book_title_prompt, model)

# Create books for different levels
create_word_document(beginner_headings, "Beginner", beginner_book_title, model)
create_word_document(intermediate_headings, "Intermediate", intermediate_book_title, model)
create_word_document(advanced_headings, "Advanced", advanced_book_title, model)



ce37473d generated
New <docx.document.Document object at 0x78de090aedd0> created
Code style applied to chapter document 
Title added to the level book
New table of contents created
Code style applied to table of contents
Title added to table of contents 
Summary Generated for I. Introduction to Large Language Models (LLMs) of Beginner's Guide to Large Language Models 
Image Generated for I. Introduction to Large Language Models (LLMs) of Beginner's Guide to Large Language Models 
Details prompt Generated for I. Introduction to Large Language Models (LLMs) of Beginner's Guide to Large Language Models 
Python code prompt Generated for I. Introduction to Large Language Models (LLMs) of Beginner's Guide to Large Language Models 
Unique identifier 1 generated
New document for the chapter created
Code style applied to the chapter document
Title added to Beginner's Guide to Large Language Models 
Heading added to I. Introduction to Large Language Models (LLMs)  
Summary added to I. Introducti